In [ ]:
%pip install faker
%pip install networkx #needed only for display of the org graph, not to generate the data

In [ ]:
#Generates mock organization data
import pandas as pd
import datetime
import random
from faker import Faker
Faker.seed(50)
 
# Initialize the data
data = []
levels = 5
numberofdivisions = 10
maxperlevel = 9
minperlevel = 5
emaildomain = '@mockorg.com'
minhiredate = datetime.date(year=1990, month=1, day=1)
maxhiredate = datetime.date(year=2024, month=10, day=10)
fake = Faker()
 
# CEO
data.append({'EmployeeID': 1, 'SupervisorID': 1, 'Level': 1, 'Name': 'Susan McCEO', 'Email': 'susan_mcceo' + emaildomain, 'DOB': '1975-08-28', 'HireDate': '2010-06-05', 'Supervisor': 'Y', 'DivisionID': 1})
df = pd.DataFrame(data)
 
orglevels = range(2,levels+1)
empid = 2
 
for orglevel in orglevels:
    prevlevel = orglevel - 1
    prevlevelrows = df[df['Level'] == prevlevel]
 
    for index, row in prevlevelrows.iterrows():
        random.seed(row['EmployeeID'])
        if orglevel == 2:
            numofreports = numberofdivisions
        else:
            numofreports = random.randint(minperlevel, maxperlevel)
        reports = range(1,numofreports+1)
        if numofreports > 0 and orglevel < levels:
            supe = 'Y'
        else:
            supe = 'N'
        
        for report in reports:
            empname = fake.unique.name()
            empemail = empname.replace(" ", "_").lower() + emaildomain

            if orglevel == 2:
                divid = empid
            else:
                divid = row['DivisionID']
            df = pd.concat([df, pd.DataFrame([{'EmployeeID': empid, 'SupervisorID': row['EmployeeID'], 'Level': orglevel, 'Name': empname, 'Email': empemail,
                'DOB':fake.date_of_birth(minimum_age = 18, maximum_age = 60), 'HireDate': fake.date_between(start_date=minhiredate, end_date=maxhiredate), 'Supervisor': supe, 'DivisionID': divid}])], ignore_index=True)
            empid = empid + 1

    print(df.shape)

# Create divisions dataframe
divisions = {'Division': ['CEO', 'Finance', 'HR', 'R&D', 'Manufacturing', 'Sales', 'Utilities', 'Marketing', 'IT', 'Quality', 'Legal', 'Operations', 'External', 'Communications', 'Discovery'], 'DivisionID': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]}
df_divs = pd.DataFrame(divisions)

# Merge/Add Division
org = pd.merge(df, df_divs, on="DivisionID")
# Display the DataFrame
display(org)
print('Final' + str(org.shape))

In [ ]:
# write org to csv
import pandas as pd
wsid = 'd6a0d49e-3d96-4d1f-8962-d98c5fe99611'
lakehouseid = '0d178ea6-89ce-40b5-aa5d-3a03f1abef14'
rowcount = str(org.shape[0])
org.to_csv(f'abfss://{wsid}@onelake.dfs.fabric.microsoft.com/{lakehouseid}/Files/MockOrg_{rowcount}.csv', index=False)

In [ ]:
#Display top 3 levels of org chart
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

#Load data and convert to Pandas
displaydf = df[df['Level']<=3]
display(displaydf)

parentcolumn = 'SupervisorID'
childcolumn = 'EmployeeID'

# Create a NetworkX directed graph from the DataFrame
G = nx.from_pandas_edgelist(displaydf, source=f'{parentcolumn}', target=f'{childcolumn}', create_using=nx.DiGraph())
# nx.draw(G, with_labels=True)
nx.draw_kamada_kawai(G, with_labels=True)
plt.show()
